In [1]:
import json, requests
import pandas as pd
#url = 'https://api.foursquare.com/v2/venues/VENUE_ID/listed'
#url = 'https://api.foursquare.com/v2/venues/explore'
url = 'https://api.foursquare.com/v2/venues/search'

params = dict(
  client_id='',  # input client id and client secret
  client_secret='',
  v='20190819',
  ll='1.335572,103.965142',
  #radius=2000, 
  sw = '1.331747, 103.961258',
  ne = '1.339397, 103.969027',  
  #VENUE_ID='HZXXY3Y',
  #intent = 'browse',  
  limit=2000
  #query='coffee',
  #limit=1
)
resp = requests.get(url=url, params=params)
data1 = json.loads(resp.text)

len(data1['response']['venues'])

198

In [2]:
list_id = []
name = []
lat = []
lng = []
postcode = []
address = []
category = []
for query in data1['response']['venues']:
    list_id.append(query['id'])
    name.append(query['name'])
    lat.append(query['location']['lat'])
    lng.append(query['location']['lng'])
    #postcode.append(query['venue']['location']['postalCode'])
    #postcode.append(query['venue']['location']['formattedAddress'][1])
    #if query['location']['postalCode']:
        #postcode.append(query['location']['postalCode'])
    #else: postcode.append('')
    address.append(query['location']['formattedAddress'])
    if query['categories']: 
        category.append(query['categories'][0]['name'])
    else: category.append('')
    #category.append(query['categories'][0]['name'] if (len(query['categories'] > 0) else: ''))

poi_data1 = pd.DataFrame({'id': list_id, 
                         'name': name,
                         'lat': lat,
                         'lng': lng,
                         #'postcode': postcode,
                         'address': address,
                         'category': category})

In [3]:
poi_data1.to_csv('poi_try1_819.csv', index=False)

In [ ]:
## OSM
## manually export from overpass.turbo
with open('export_sep_10_try3.geojson') as f:
    data = json.load(f)
    
list_id = []
name = []
lat = []
lng = []
postcode = []
address = []
category = []
for query in data['features']:
    list_id.append(query['id'])
    #if query['properties']['name']: 
        #name.append(query['properties']['name'])
    #else: name.append('')
    try: name.append(query['properties']['name'])  
    except: name.append('')
    lat.append(query['geometry']['coordinates'][1])
    lng.append(query['geometry']['coordinates'][0])
    #postcode.append(query['venue']['location']['postalCode'])
    #postcode.append(query['venue']['location']['formattedAddress'][1])
    #if query['location']['postalCode']:
        #postcode.append(query['location']['postalCode'])
    #else: postcode.append('')
    #address.append(query['location']['formattedAddress'])
    if 'highway' in query['properties'].keys(): 
        category.append(query['properties']['highway'])
    elif 'barrier' in query['properties'].keys(): 
        category.append(query['properties']['barrier'])
    elif 'shop' in query['properties'].keys(): 
        category.append(query['properties']['shop'])
    elif 'public_transport' in query['properties'].keys(): 
        category.append(query['properties']['public_transport'] + '(public_transport)')
    elif 'amenity' in query['properties'].keys(): 
        category.append(query['properties']['amenity'])
    elif 'place' in query['properties'].keys(): 
        category.append(query['properties']['place'])
    elif 'station' in query['properties'].keys(): 
        category.append(query['properties']['station'] + '_station')
    elif 'building' in query['properties'].keys(): 
        category.append(query['properties']['building'])
    elif 'tourism' in query['properties'].keys(): 
        category.append(query['properties']['tourism'])
    elif 'office' in query['properties'].keys():
        if query['properties']['office'] == 'yes': 
            category.append('office')
    elif 'railway' in query['properties'].keys(): 
        category.append(query['properties']['railway'] + '(railway)') # overlap with public transport?
    elif 'crossing' in query['properties'].keys(): 
        category.append(query['properties']['crossing'])
    elif 'natural' in query['properties'].keys(): 
        category.append(query['properties']['natural'])
    elif 'location' in query['properties'].keys(): 
        category.append(query['properties']['location'])
    elif 'leisure' in query['properties'].keys(): 
        category.append(query['properties']['leisure'])
    elif 'aeroway' in query['properties'].keys(): 
        category.append(query['properties']['aeroway'] + '(aeroway)')
    elif 'entrance' in query['properties'].keys(): 
        if query['properties']['entrance'] == 'yes': 
            category.append('entrance')
            
    else: category.append('')
    #category.append(query['categories'][0]['name'] if (len(query['categories'] > 0) else: ''))

poi_data = pd.DataFrame({'id': list_id, 
                         'name': name,
                         'lat': lat,
                         'lng': lng,
                         #'postcode': postcode,
                         #'address': address,
                         'category': category})

In [ ]:
#poi_data.merge(poi_data3, on = ['lat','lng'], how = 'inner')
#poi_data3['lat'] = poi_data3['lat'].apply(lambda x: round(x,6))
#poi_data3['lng'] = poi_data3['lng'].apply(lambda x: round(x,6))

In [ ]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
#!pip install python-Levenshtein
import Levenshtein
import distance
from fuzzywuzzy import fuzz

#similar('Xilin', 'Xilinx Building')
#Levenshtein.ratio('Xilin', 'Xilinx Building')
#1-distance.jaccard('Xilin', 'Xilinx Building')
#fuzz.ratio('Xilin', 'Xilinx Building')
#fuzz.token_sort_ratio('Xilin', 'Xilinx Building')
#fuzz.token_set_ratio('Xilin', 'Xilinx Building')

In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [ ]:
def match_latlon_and_name(name, lat, lon, df2, min_score=0, dist_thres=0.03):
    # -1 score incase we don't get any matches
    max_score = -1
    # Returning empty name for no match as well
    max_name = ""
    max_lat = ""
    max_lon = ""
    # Iternating over all names in the other
    for row in df2.iterrows():
        name2 = row[1][1]
        lat2 = row[1][2]
        lon2 = row[1][3]
        #Finding fuzzy match score
        score = fuzz.ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score) & (haversine(lon, lat, lon2, lat2) < dist_thres) & (score<100):
            max_name = name2
            max_lat = lat2
            max_lon = lon2
            max_score = score
    return (max_name, max_lat, max_lon, max_score)

In [ ]:
dict_list = []
# iterating over our players without salaries found above
for row in poi_data3.iterrows():
    name = row[1][1]
    lat = row[1][2]
    lon = row[1][3]
    # Use our method to find best match, we can set a threshold here
    match = match_latlon_and_name(name, lat, lon, poi_data, 55, 0.1)
    
    # New dict for storing data
    dict_ = {}
    dict_.update({"foursquare_name" : name})
    dict_.update({"foursquare_lat" : lat})
    dict_.update({"foursquare_lon" : lon})
    dict_.update({"osm_name" : match[0]})
    dict_.update({"osm_lat" : match[1]})
    dict_.update({"osm_lon" : match[2]})
    dict_.update({"score" : match[3]})
    dict_list.append(dict_)
    
merge_table = pd.DataFrame(dict_list)
# Display results
merge_table